In [ ]:
import pandas as pd
!pip install transformers

     |████████████████████████████████| 4.0 MB 12.3 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 880 kB 49.9 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=807d6839b7336460dce251c4b94e19301306513af3128481859fea9f7700f691
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1HABJpbnnFW_oW6u8yrkXbFFFVMn5JvsN/view?usp=sharing' 

In [ ]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Dec_VADER.csv') 
 
df = pd.read_csv('Train_Dec_VADER.csv')
print(df)

       Unnamed: 0                                               text  \
0               0   They started the armed conflict after Obama/B...   
1               1  Tucker Carlson emphasized on how much Ukraine ...   
2               2  Since day one, the military fighting against U...   
3               3  A notable Russian OSS developer promotes a pro...   
4               4  US believes Russia could invade Ukraine ‘as so...   
...           ...                                                ...   
56126       56126   Russia could roll through Ukraine in a matter...   
56127       56127  Russia Preparing 175K Troops For Ukraine Offen...   
56128       56128   In other words, Russia is Male and Ukraine is...   
56129       56129  How the US Military Could Mobilize If Russia I...   
56130       56130  “The U.S. and NATO reject Putin’s demands that...   

       Compound  Positive  Negative  Neutral Sentiment  
0       -0.7902     0.000     0.174    0.826  Negative  
1        0.2003     0

In [ ]:
df

,Unnamed: 0,text,Compound,Positive,Negative,Neutral,Sentiment
0,0,They started the armed conflict after Obama/B...,-0.7902,0.000,0.174,0.826,Negative
1,1,Tucker Carlson emphasized on how much Ukraine ...,0.2003,0.082,0.051,0.866,Positive
2,2,"Since day one, the military fighting against U...",-0.0516,0.047,0.051,0.902,Negative
3,3,A notable Russian OSS developer promotes a pro...,0.0258,0.095,0.093,0.812,Neutral
4,4,US believes Russia could invade Ukraine ‘as so...,0.0000,0.000,0.000,1.000,Neutral
...,...,...,...,...,...,...,...
56126,56126,Russia could roll through Ukraine in a matter...,0.0258,0.061,0.000,0.939,Neutral
56127,56127,Russia Preparing 175K Troops For Ukraine Offen...,0.0258,0.181,0.175,0.643,Neutral
56128,56128,"In other words, Russia is Male and Ukraine is...",0.0000,0.000,0.000,1.000,Neutral
56129,56129,How the US Military Could Mobilize If Russia I...,0.0000,0.000,0.000,1.000,Neutral


In [ ]:
df = df.drop(['Unnamed: 0','Compound', 'Positive','Negative','Neutral'], axis=1)

In [ ]:
df = df.replace('Negative', 0)
df = df.replace('Neutral', 1)
df = df.replace('Positive', 2)

In [ ]:
df

,text,Sentiment
0,They started the armed conflict after Obama/B...,0
1,Tucker Carlson emphasized on how much Ukraine ...,2
2,"Since day one, the military fighting against U...",0
3,A notable Russian OSS developer promotes a pro...,1
4,US believes Russia could invade Ukraine ‘as so...,1
...,...,...
56126,Russia could roll through Ukraine in a matter...,1
56127,Russia Preparing 175K Troops For Ukraine Offen...,1
56128,"In other words, Russia is Male and Ukraine is...",1
56129,How the US Military Could Mobilize If Russia I...,1


In [ ]:
#80,10,10 split
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=1)

test_set, valid_set = train_test_split(test_set, test_size=0.5, random_state=1)

In [ ]:
#train_set = train_set.sample(frac=0.8,random_state=1)

In [ ]:
#valid_set = valid_set.sample(frac=0.8,random_state=1)

In [ ]:
#test_set = test_set.sample(frac=0.8,random_state=1)

In [ ]:
from google.colab import drive
drive.mount('drive')
import sys
sys.path.append('drive/My Drive/Colab Notebooks')

Mounted at drive


In [ ]:
#code from NLPiation

import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

# The dataset class
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews    = reviews
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.reviews)
  
    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens    = True,
            max_length            = 128,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = 'pt',
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_set_dataset = TheDataset(
    reviews    = train_set.text.tolist(),
    sentiments = train_set.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

valid_set_dataset = TheDataset(
    reviews    = valid_set.text.tolist(),
    sentiments = valid_set.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained("bert-large-uncased", problem_type="single_label_classification", num_labels=3)

# Freeze the whole BERT model and train just the classifier
#for name, param in model.bert.named_parameters():
    #param.requires_grad = False

# Freeze BERT except the pooler layer
#for name, param in model.bert.named_parameters():
    #if not name.startswith('pooler'):
        #param.requires_grad = False

# Freeze the first 18 layers of the BERT
for name, param in model.bert.named_parameters():
    if ( not name.startswith('pooler') ) and "layer.23" and "layer.22" and "layer.21" and "layer.20" and "layer.19" and "layer.18" not in name :
        param.requires_grad = False

# The function to get the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

    

# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = 'drive/My Drive/Colab Notebooks',
    num_train_epochs            = 8,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 64,
    warmup_steps                = 10000,
    learning_rate               = 5e-5,
    weight_decay                = 0.01,
    adam_beta1                  = 0.9,
    adam_beta2                  = 0.999,
    save_strategy               = "epoch",
    evaluation_strategy         = "epoch",
    eval_steps                  = 1,
    #save_total_limit            = 5,
    #metric_for_best_model       = 'eval_loss',
    load_best_model_at_end      =True
)

# Define the Huggingface Trainer object
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_set_dataset,
    eval_dataset    = valid_set_dataset,
    compute_metrics = compute_metrics,
    callbacks       = [EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start pre-training!
trainer.train()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.708700,0.633610,0.740114,0.706705,0.725158,0.698243
2,0.525700,0.522912,0.796402,0.770499,0.813751,0.746894
3,0.435500,0.432840,0.837015,0.822058,0.830084,0.817245
4,0.370800,0.399408,0.848058,0.835115,0.852177,0.822146
5,0.281400,0.435063,0.856252,0.845021,0.862467,0.833629
6,0.200300,0.497386,0.861062,0.849940,0.866649,0.837528
7,0.162600,0.603449,0.859637,0.847705,0.863931,0.835867


***** Running Evaluation *****
  Num examples = 5614
  Batch size = 64
Saving model checkpoint to drive/My Drive/Colab Notebooks/checkpoint-2807
Configuration saved in drive/My Drive/Colab Notebooks/checkpoint-2807/config.json
Model weights saved in drive/My Drive/Colab Notebooks/checkpoint-2807/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5614
  Batch size = 64
Saving model checkpoint to drive/My Drive/Colab Notebooks/checkpoint-5614
Configuration saved in drive/My Drive/Colab Notebooks/checkpoint-5614/config.json
Model weights saved in drive/My Drive/Colab Notebooks/checkpoint-5614/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5614
  Batch size = 64
Saving model checkpoint to drive/My Drive/Colab Notebooks/checkpoint-8421
Configuration saved in drive/My Drive/Colab Notebooks/checkpoint-8421/config.json
Model weights saved in drive/My Drive/Colab Notebooks/checkpoint-8421/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5614
  

TrainOutput(global_step=19649, training_loss=0.41465324039974677, metrics={'train_runtime': 6514.6451, 'train_samples_per_second': 68.928, 'train_steps_per_second': 4.309, 'total_flos': 7.32332778320425e+16, 'train_loss': 0.41465324039974677, 'epoch': 7.0})

In [ ]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("drive/My Drive/Colab Notebooks/checkpoint-11228", problem_type="single_label_classification", num_labels=3)

# Make the test set ready
test_set_dataset = TheDataset(
    reviews    = test_set.text.tolist(),
    sentiments = test_set.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = 'drive/My Drive/Colab Notebooks',
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics =compute_metrics,
)

PredictionOutput = trainer.predict(test_set_dataset)

loading configuration file drive/My Drive/Colab Notebooks/checkpoint-11228/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loadi

In [ ]:
import numpy as np
PredictionOutput

PredictionOutput(predictions=array([[ 1.1162671 ,  1.4280463 , -1.9198462 ],
       [-1.3157207 , -1.5286156 ,  2.2701845 ],
       [ 4.0374575 , -1.9758567 , -1.9382817 ],
       ...,
       [-0.5617279 ,  1.0945756 , -0.07932599],
       [ 1.8442013 , -1.8872157 , -0.09770372],
       [-1.1963207 , -1.9948266 ,  2.4621437 ]], dtype=float32), label_ids=array([1, 2, 0, ..., 1, 0, 2]), metrics={'test_loss': 0.398332417011261, 'test_accuracy': 0.8555139853910565, 'test_f1': 0.8413599715379219, 'test_precision': 0.8604304139707325, 'test_recall': 0.8268557990533084, 'test_runtime': 78.7899, 'test_samples_per_second': 71.24, 'test_steps_per_second': 8.91})

In [ ]:
predictions, label_ids, metrics = PredictionOutput

In [ ]:
metrics

{'test_balanced_accuracy': 0.7978215164846297,
 'test_f1': 0.7996179457912057,
 'test_loss': 0.5070290565490723,
 'test_precision': 0.8017353967317038,
 'test_recall': 0.7978215164846297,
 'test_runtime': 1147.1624,
 'test_samples_per_second': 9.787,
 'test_steps_per_second': 1.224}

In [ ]:
df1 = pd.DataFrame(metrics, index=['Output'])

In [ ]:
df1

,test_loss,test_balanced_accuracy,test_f1,test_precision,test_recall,test_runtime,test_samples_per_second,test_steps_per_second
Output,0.507029,0.797822,0.799618,0.801735,0.797822,1147.1624,9.787,1.224


In [ ]:
df2 = pd.DataFrame(predictions)

In [ ]:
df2

,0,1,2
0,-2.096037,1.464743,0.665169
1,3.632438,-2.071273,-2.919343
2,0.936616,-1.865878,0.049600
3,1.543650,-1.613585,-0.860745
4,-3.516509,4.259713,-1.870149
...,...,...,...
11222,-1.398229,-1.450295,2.285965
11223,-3.191445,-3.537298,7.063457
11224,2.085979,-1.913138,-0.325679
11225,5.274303,-3.780576,-1.656197


In [ ]:
test_set

,text,Sentiment
43963,Biden admin is handing Ukraine over to Russia ...,2
41962,We know Trump held Putin at bay because Russi...,0
20660,Russia is not a credible threat to our No ...,0
45182,The more NATO moves their fence closer to Rus...,0
696,"Putin-Biden video call. 3 reasons why ""Russia ...",1
...,...,...
40518,What about Ukraine and Russia's other neighbo...,2
18759,"If Russia invades Ukraine, good luck - your o...",2
16105,"Dr. David Donat Cattin, Secretary-General of ,...",0
35183,russia: US warns Russia of crippling sanctions...,0
